In [5]:
import os
import pandas as pd
from shutil import copy
import glob
import numpy as np
import torch

# Read-in Represenations of Image Data

In [12]:
# access dataset 
RELATIVE_PATH_SUPCON = r'../SupConFeatureRepresentations/'
Z_FILES = glob.glob(RELATIVE_PATH_SUPCON + "/*z.csv")
Z_FILES

# 128 vectors
data = []
for f in Z_FILES:
    print(f)
    df = pd.read_csv(f, sep=',', error_bad_lines=False)
    df = df[['tweet_id', 'image_id', 'actual label', 'features']]
    df['features'] = df['features'].apply(lambda x: np.fromstring(x.replace("|", " "), sep=' '), 0)
    data.append(df)
    
img_val, img_test, img_train = data

../SupConFeatureRepresentations/val_z.csv
../SupConFeatureRepresentations/test_z.csv
../SupConFeatureRepresentations/train_z.csv


In [13]:
x, y,z = len(img_train), len(img_val), len(img_test)

In [14]:
print(x,y,z)

11878 2464 2576


In [7]:
val.head()

,tweet_id,image_id,actual label,features
0,909474490870059014,909474490870059014_0,0,"[-0.096236, -0.158864, -0.022322, 0.098831, 0...."
1,906844870538665984,906844870538665984_0,0,"[-0.096429, -0.166481, -0.019935, 0.095985, 0...."
2,909435887200718848,909435887200718848_0,1,"[-0.09579, -0.166469, -0.018783, 0.098647, 0.0..."
3,909458363850862592,909458363850862592_0,1,"[-0.097234, -0.168602, -0.016127, 0.100168, 0...."
4,912962261622378496,912962261622378496_0,1,"[-0.092785, -0.163104, -0.016705, 0.099005, 0...."


# Read-in Text Representation

In [126]:
RELATIVE_PATH_EMB = r'../data/final_embeddings_whole_dataset.csv'
tweet_emb_df = pd.read_csv(RELATIVE_PATH_EMB , sep=',', error_bad_lines=False)
tweet_emb_df.head()
embeddings = pd.read_csv("pd_all_emb.csv", sep=',',header=None).head(2)

In [128]:
embeddings = pd.read_csv("pd_all_emb.csv", sep=',')
embeddings.head(2)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.046911,0.104039,0.080677,0.130218,0.138316,0.065321,0.057658,0.062567,0.106187,0.041806,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.048901,0.109742,0.091063,0.120951,0.155213,0.076462,0.053102,0.083131,0.100610,0.068814,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [129]:
final_emb = list()
for i in range(len(embeddings.values)):
    final_emb.append(embeddings.values[i])

In [130]:
tweet_emb_df["emb"] = final_emb

In [131]:
ltrain, lvalid, ltest = 8933,1914,1915
leval = ltrain + lvalid
tweet_train = tweet_emb_df[:ltrain]
tweet_val   = tweet_emb_df[ltrain:leval]
tweet_test  = tweet_emb_df[leval:]

## Match Images and Tweets

In [133]:
test_data = pd.merge(tweet_test,img_test, on="tweet_id")
val_data = pd.merge(tweet_val,img_val, on="tweet_id")
train_data = pd.merge(tweet_train,img_train, on="tweet_id")

In [191]:
train_fea = train_data.features.values
train_emb = train_data.emb.values

joint_features_train = []
for i,j in zip(train_fea,train_emb):
    z = np.array(list(i) +list(j))
    joint_features_train.append(z)
    
train_data["joint"]= joint_features_train

train_fdatat = train_data[['tweet_id',  'text_info','image_id', 'joint']]

train_data.to_csv("train_data_joint_rep.csv",index=False)

In [200]:
train_data.columns

Index(['tweet_id', 'text_info_conf', 'tweet_text', 'text_info', 'input_ids',
       'attention', 'label_np', 'embedding', 'id_check', 'emb', 'image_id',
       'actual label', 'features', 'joint'],
      dtype='object')

In [203]:
train_data = train_data.drop(columns=[ 'text_info_conf', 'tweet_text', 'text_info', 'input_ids',
       'attention', 'label_np', 'embedding', 'id_check', 'emb', 'features',])

In [205]:

train_data.to_csv("train_data_joint_rep.csv",index=False)

In [204]:
train_data.head(2)

,tweet_id,image_id,actual label,joint
0,923142910958333952,923142910958333952_0,0,"[-0.09367, -0.163019, -0.01667, 0.098565, 0.03..."
1,930044157699280896,930044157699280896_0,1,"[-0.095589, -0.169625, -0.022822, 0.09779, 0.0..."


In [185]:
val_fea = val_data.features.values
val_emb = val_data.emb.values

joint_features_val = []
for i,j in zip(val_fea,val_emb):
    z = np.array(list(i) +list(j))
    joint_features_val.append(z)
    
val_data["joint"]= joint_features_val

val_data = val_data[['tweet_id',  'text_info','image_id', 'joint']]

val_data.to_csv("val_data_joint_rep.csv",index=False)

In [189]:
val_data.head(2)

,tweet_id,text_info,image_id,joint
0,909474490870059014,0,909474490870059014_0,"[-0.096236, -0.158864, -0.022322, 0.098831, 0...."
1,906844870538665984,0,906844870538665984_0,"[-0.096429, -0.166481, -0.019935, 0.095985, 0...."


In [186]:
test_fea = test_data.features.values
test_emb = test_data.emb.values

joint_features_test = []
for i,j in zip(test_fea,test_emb):
    z = np.array(list(i) +list(j))
    joint_features_test.append(z)
    
test_data["joint"]= joint_features_test

test_data =test_data[['tweet_id',  'text_info','image_id', 'joint']]

test_data.to_csv("test_data_joint_rep.csv",index=False)

AttributeError: 'DataFrame' object has no attribute 'features'

In [208]:
test_data.head(2)

,tweet_id,text_info,image_id,joint
0,912342960682831872,1,912342960682831872_0,"[-0.094228, -0.166212, -0.018292, 0.098913, 0...."
1,909869734216257544,1,909869734216257544_0,"[-0.095283, -0.164992, -0.018444, 0.097539, 0...."


In [188]:
val_data

,tweet_id,text_info,image_id,joint
0,909474490870059014,0,909474490870059014_0,"[-0.096236, -0.158864, -0.022322, 0.098831, 0...."
1,906844870538665984,0,906844870538665984_0,"[-0.096429, -0.166481, -0.019935, 0.095985, 0...."
2,909435887200718848,1,909435887200718848_0,"[-0.09579, -0.166469, -0.018783, 0.098647, 0.0..."
3,909458363850862592,1,909458363850862592_0,"[-0.097234, -0.168602, -0.016127, 0.100168, 0...."
4,912962261622378496,1,912962261622378496_0,"[-0.092785, -0.163104, -0.016705, 0.099005, 0...."
...,...,...,...,...
2727,927941985155141638,0,927941985155141638_0,"[-0.091227, -0.164622, -0.017013, 0.092714, 0...."
2728,912417890770579457,1,912417890770579457_0,"[-0.095551, -0.163043, -0.016834, 0.102889, 0...."
2729,914148708559630336,1,914148708559630336_0,"[-0.091152, -0.16564, -0.019326, 0.101932, 0.0..."
2730,914148708559630336,1,914148708559630336_1,"[-0.095746, -0.166874, -0.018904, 0.097426, 0...."


In [231]:
128 for img, 
train_data.joint.values[0].shape

(192,)

In [226]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, features, labels):
        'Initialization'
        self.labels = labels
        self.features = features

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.features)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample

        # Load data and get label
        X = torch.features[index]
        y = self.labels[index]

        return X, y

In [227]:
train = Dataset(train_data["joint"],train_data["actual label"])
test = Dataset(test_data["joint"],test_data["text_info"])
BATCH_SIZE =32

In [228]:
train_dataloader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE,shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE,shuffle=True)